In [18]:
import pandas as pd
import numpy as np
import tensorflow as tf
from time_series_transform.tensorflow_adsopter import *
from time_series_transform.sequence_transfomer import *
from time_series_transform.time_series_transformer import Time_Series_Transformer

ModuleNotFoundError: No module named 'time_series_transform.tensorflow_adsopter'

In [2]:
df = pd.read_csv('../Data/Train/sales_train_validation.csv').sample(2)
dimList = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']
df = df.drop(dimList+['id'],axis =1)
dimList = []

In [12]:
for i in df:
    df[i] = df[i].astype(np.float32)

In [16]:
fe = Time_Series_Transformer(df,[],seqTransformerList = [])
gen = fe.np_to_time_tensor_generator(5)

In [14]:
tfg = TFRecord_Generator("tfg.tfRecord")
tfg.write_tfRecord(gen)